# zib gereed maken voor mapping

* inlezen en layout consistent maken met XtEHR
* opslaan voor mapping - zie mapping Notebook

## imports

In [1]:
import pandas as pd
import os

## toon de zibs in folder (hier: zib2024)

In [2]:
base = 'zibs2024'
folder = os.path.join(base, 'downloads')
files = os.listdir(folder)
files

['nl.zorg.AdministrationAgreement-v3.0(2024EN).xlsx',
 'nl.zorg.DispenseRequest-v4.0(2024EN).xlsx',
 'nl.zorg.HealthcareProvider-v3.6(2024EN).xlsx',
 'nl.zorg.MedicationAdministration2-v3.0(2024EN).xlsx',
 'nl.zorg.MedicationAgreement-v4.0(2024EN).xlsx',
 'nl.zorg.MedicationDispense-v3.0.1(2024EN).xlsx',
 'nl.zorg.MedicationUse2-v3.0(2024EN).xlsx',
 'Nl.zorg.part.PharmaceuticalProduct-v2.3(2024EN).xlsx']

## kies een zib

In [3]:
file = files[7] # Selecting the n-th file in the list (run trough them one by one)
print(file)
url = os.path.join(folder, file)
zibdf = pd.read_excel(url, sheet_name='Data', header=1)
zibdf

Nl.zorg.part.PharmaceuticalProduct-v2.3(2024EN).xlsx


C:\Users\engel_nict1\AppData\Roaming\Python\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,Concept,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Alias,Type,Card.,Stereotype,Id,Definition,DefinitionCode,Reference,Constraints
0,NaN,PharmaceuticalProduct,NaN,NaN,NaN,NaN,NaN,NL: FarmaceutischProduct,NaN,NaN,rootconcept,NL-CM:9.7.19926,Root concept of the PharmaceuticalProduct part...,NaN,NaN,NaN
1,NaN,NaN,MedicationCode,NaN,NaN,NaN,NaN,NL: ProductCode,CD,0..1,data,NL-CM:9.7.19927,Coding medication in the Netherlands is done o...,NaN,ProductCodeATCCodelist,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ProductCodeGPKCodelist,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ProductCodeGTINCodelist,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ProductCodeHPKCodelist,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ProductCodePRKCodelist,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ProductCodeSNOMEDCTCodelist,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ProductCodeZICodelist,NaN
8,NaN,NaN,ProductSpecifications,NaN,NaN,NaN,NaN,NL: ProductSpecificatie,NaN,0..1,container,NL-CM:9.7.19928,Container of the ProductSpecifications concept...,NaN,NaN,NaN
9,NaN,NaN,NaN,Batch/lotNumber,NaN,NaN,NaN,NL: Batch/lotNummer,ST,0..1,data,NL-CM:9.7.22279,The batch/lot number provides a unique identif...,NaN,NaN,NaN


## opschonen zib

* lowercase kolomnamen
* paar labels laten vervallen
* name en path kolom toevoegen
* name vullen (naam attribuut)
* kolommen opschonen tot benodigde

Handmatig:
* path vullen (als in logical model, dus pad.pad2.pad3.naam etc.)

In [4]:
zibdf.columns = [col.lower() for col in zibdf.columns]
zibdf = zibdf.drop(labels=['unnamed: 0', 'reference'], axis=1).dropna(how='all')
zibdf.insert(0, 'name', None)
zibdf.insert(0, 'path', None)
zibdf.fillna('')
concepts = []
for i, row in zibdf.iterrows():
    # Loop thru columns 2 tot 8 (Concept and 'Unnamed' ones)
    for j in range(2, 8):
        # print(j, row.iloc[j])
        if not pd.isna(row.iloc[j]):
            # Remove last element if next column is empty
            if len(concepts) > j -2:
                concepts.pop()
            try:
                concepts[j - 2] = row.iloc[j]
            except IndexError:
                concepts.append(row.iloc[j])
    # print(i, concepts)
    zibdf.at[i, 'path'] = '.'.join(concepts)
    zibdf.at[i, 'name'] = concepts[-1]
zibdf = zibdf[['name', 'path', 'alias', 'type', 'card.', 'stereotype', 'id', 'definition', 'definitioncode']].fillna('')
zibdf

,name,path,alias,type,card.,stereotype,id,definition,definitioncode
0,PharmaceuticalProduct,PharmaceuticalProduct,NL: FarmaceutischProduct,,,rootconcept,NL-CM:9.7.19926,Root concept of the PharmaceuticalProduct part...,
1,MedicationCode,PharmaceuticalProduct.MedicationCode,NL: ProductCode,CD,0..1,data,NL-CM:9.7.19927,Coding medication in the Netherlands is done o...,
8,ProductSpecifications,PharmaceuticalProduct.ProductSpecifications,NL: ProductSpecificatie,,0..1,container,NL-CM:9.7.19928,Container of the ProductSpecifications concept...,
9,Batch/lotNumber,PharmaceuticalProduct.ProductSpecifications.Ba...,NL: Batch/lotNummer,ST,0..1,data,NL-CM:9.7.22279,The batch/lot number provides a unique identif...,
10,SerieNumber,PharmaceuticalProduct.ProductSpecifications.Se...,NL: SerieNummer,ST,0..1,data,NL-CM:9.7.22280,The serial number uniquely identifies the sour...,
11,PharmaceuticalForm,PharmaceuticalProduct.ProductSpecifications.Ph...,NL: FarmaceutischeVorm,CD,0..1,data,NL-CM:9.7.19931,The pharmaceutical form indicates the form of ...,
12,Medication,PharmaceuticalProduct.ProductSpecifications.Me...,NL: ProductNaam,ST,0..1,data,NL-CM:9.7.19929,"For medication which has no code, enter the co...",
13,Description,PharmaceuticalProduct.ProductSpecifications.De...,NL: Omschrijving,ST,0..1,data,NL-CM:9.7.19784,A textual description of the type of medicatio...,
14,Ingredient,PharmaceuticalProduct.ProductSpecifications.In...,NL: Ingredient,,0..*,container,NL-CM:9.7.19932,Container of the Ingredient concept. This cont...,
15,SubstanceCode,PharmaceuticalProduct.ProductSpecifications.In...,NL: IngredientCode,CD,0..1,data,NL-CM:9.7.19934,"Active substance or excipient. \nHere, the sam...",


## Excel opslaan

In [6]:
url = os.path.join(base, 'mappable', file)
zibdf.to_excel(url, index=False)